<a href="https://colab.research.google.com/github/jkweh/ganart/blob/master/Train_and_Generate_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

---



StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

# 1. Clone repos to Google Drive

---

First, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

%cd /content/drive/MyDrive/

if not os.path.isdir("/content/drive/MyDrive/git"):
  !mkdir /content/drive/MyDrive/git
%cd git

stylegan2_gitdir="/content/drive/MyDrive/git/stylegan2-ada"
if not os.path.isdir("/content/drive/MyDrive/git/stylegan2-ada"):
  !git clone https://github.com/dvschultz/stylegan2-ada
else:
  %cd "{stylegan2_gitdir}"
  !git pull origin master

ganart_gitdir="/content/drive/MyDrive/git/ganart"
if not os.path.isdir({ganart_gitdir}):
  !git clone https://github.com/jkweh/ganart
else:
  %cd "{ganart_gitdir}"
  !git pull origin master



And then finally create the recommended directory tree

In [ ]:
base_path = "/content/drive/MyDrive"
project_name=""

dataset_dir=f"{base_path}/datasets/{project_name}"
!mkdir -p {dataset_dir}

training_dir=f"{base_path}/gan/training/{project_name}"
!mkdir -p {training_dir}

outdir=f"{base_path}/gan/data/outdir/{project_name}"
!mkdir -p {outdir}

# 2. Scrape and pre-process
---


## Run the scraper

In [ ]:
import os
if os.path.isdir({ganart_gitdir}):
  %cd {ganart_gitdir}
  !python3 ./scrape-wikiart-by-genre.py --outdir {outdir}
  dl_dir=f"{outdir}/dl"

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to continue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
!tar -zxvf /content/drive/MyDrive/art.tar.gz

Now that your image dataset is uploaded, we need to convert it to the `tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
%cd stylegan2_gitdir

tfrecords_path = f"{training_dir}/tfrecords"
!python dataset_tool.py create_from_images {dataset_dir} {tfrecords_path}

# 3. Train a custom model
---
We’re ready to start training!**bold text**

In [ ]:
%cd {stylegan2_gitdir}

results_path = f"{training_dir}/results"

# This name must EXACTLY match the dataset name you used when creating the .tfrecords file.
snapshot_count = 2  # How often should the model generate samples and a .pkl file.
mirrored = True  # should the images be mirrored left to right?
mirroredY = True  # Should the images be mirrored top to bottom?
metric_list = "fid50k_full"  # Metrics?

resume_from = "ffhq1024"  # Running it for the first time? Set to ffhq(+resolution), e.g. `ffhq1024`.
# resume_from = f"{results_path}/network-snapshot-000008.pkl"  # Resuming? Get the path to your latest .pkl file and use that.

!python train.py --outdir {results_path} --snap={snapshot_count} --cfg=11gb-gpu --data={tfrecords_path} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}

### While it’s training...

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

# 4. Generate!

---



In [ ]:
!pip install -U opensimplex
%tensorflow_version 1.x

In [ ]:
%cd {stylegan2_gitdir}
network_file = f"{results_path}/network-snapshot-000008.pkl"
!python ./generate.py generate-images --outdir=f"{outdir}/images" --seeds="1000-1050" --network={network_file}

# 5. Post-Process

---



In [ ]:
!pip install -U ISR
# There's some weird stuff going on that causes an error; this usually fixes it.
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
%cd /content/drive/MyDrive/git/ganart
!git pull
!python ./post_process_images.py --indir f"{outdir}/images"